In [17]:
from dotenv import load_dotenv
from pydantic import BaseModel
from langchain_ai21 import ChatAI21
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent, AgentExecutor

In [3]:
load_dotenv()

True

In [4]:
llm = ChatAI21(
    model="jamba-mini"
)

In [5]:
response = llm.invoke("what is the capital of France?")

In [9]:
print(response)

content='The capital of France is **Paris**.' additional_kwargs={} response_metadata={} id='run-d59a9013-0009-420d-8ddd-02c4db093cc2-0'


In [10]:
class ResearchResponse(BaseModel):
    topic: str
    summary: str
    sources: list[str]
    tools_used: list[str]

In [12]:
parser = PydanticOutputParser(pydantic_object=ResearchResponse)

In [52]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", 
         """
         You are a research assistant that will help generate a research paper
         Answer the query and use necessary tools
         Warp the output in this format and no premamble and other formats\n{format_instructions}
         """,
         ),
        ("placeholder","{chat_history}"),
        ("human","{query}"),
        ("placeholder","{agent_scratchpad}"),
    ]
).partial(format_instructions=parser.get_format_instructions())

In [53]:
agent = create_tool_calling_agent(
    llm=llm,
    prompt=prompt,
    tools=[],
)

In [54]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=[],
    verbose=True,
)

In [55]:
raw_response = agent_executor.invoke({
    "query": "What is the capital of France?",
})



> Entering new AgentExecutor chain...
The capital of France is Paris.

> Finished chain.


In [56]:
print(raw_response)

{'query': 'What is the capital of France?', 'output': 'The capital of France is Paris.'}


In [44]:
structured_response = parser.parse(raw_response.content)


AttributeError: 'dict' object has no attribute 'content'